In [1]:
import sys
import os
parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)
import system.robot as robotSystem
import casadi as ca
import system.utils.transformation_matrix as Transformation

In [2]:
alpha = robotSystem.RobotDynamics()
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
path_to_urdf = os.path.join(
    project_root,
    'usage',
    'urdf','reach_alpha_5',
    'alpha_5_robot.urdf'
)
alpha.from_file(path_to_urdf)
root = "base_link"
tip = "alpha_standard_jaws_base_link"
kinematic_dict, K, P, L, D, C, g, B, qdd, joint_torque, sys_id_coeff, F_payload = alpha.build_model(root, tip, floating_base=True)
kinematic_dict = alpha.get_kinematic_dict()
c_parms, m_params, I_params, fv_coeff, fc_coeff, fs_coeff, v_s_coeff, vec_g, r_com_body, m_p, q, q_dot, q_dotdot, tau, base_pose, world_pose, tip_offset_pose = kinematic_dict['parameters']
n_joints = alpha.get_n_joints(root, tip)

In [3]:
# decision variables for IK
x_world_dot = ca.SX.sym('x_world_dot', 6, 1)     # [vx,vy,vz, roll_dot,pitch_dot,yaw_dot]
q_dot_cmd   = ca.SX.sym('q_dot_cmd', n_joints, 1)
nu = ca.vertcat(x_world_dot, q_dot_cmd)          # (6+n) x 1

# position task
Jp = kinematic_dict['geo_J_full'][-1][0:3, :]    # 3 x (6+n)
p  = kinematic_dict['Fks'][-1][0:3]              # 3 x 1

kp = ca.SX.sym('kp', 3, 1)
p_des = ca.SX.sym('p_des', 3, 1)

# desired end-effector linear velocity (per-axis gains)
v_des = ca.diag(kp) @ (p_des - p)                # 3 x 1

# task residual: match task-space linear velocity
e_p_task = Jp @ nu - v_des                       # 3 x 1

# tool axis in tool frame (choose one)
a_tool = ca.DM([0, 0, 1])     # z-axis of end effector frame
# a_tool = ca.DM([1, 0, 0])   # x-axis alternative
# a_tool = ca.DM([0, 1, 0])   # y-axis alternative

# desired axis in world frame (unit vector)
a_des = ca.SX.sym('a_des', 3, 1)                 # keep symbolic for function input
a_des_u = a_des / ca.norm_2(a_des)   # normalized copy, use this in computations


R_ee = kinematic_dict['R_symx'][-1]   # 3x3
a_now = R_ee @ a_tool                                   # 3x1, tool axis expressed in world

e_axis = ca.cross(a_now, a_des_u)        # 3x1, zero when aligned
k_axis = ca.SX.sym('k_axis')           # gain (rad/s per unit error)
w_des_axis = -k_axis * e_axis          # 3x1 desired angular velocity in world

Jw = kinematic_dict['geo_J_full'][-1][3:6, :]   # 3 x (6+n), spatial angular part                             # 3x1
e_axis_task = Jw @ nu - w_des_axis               # 3x1

# roll,pitch soft stabilization (servo Euler angles back toward 0)
phi   = world_pose[3]
theta = world_pose[4]
psi = world_pose[5]
k_rp = ca.SX.sym('k_rp')                         # scalar

# desired Euler rates (stabilize roll, pitch, leave yaw free)
phi_dot_des   = -k_rp * phi
theta_dot_des = -k_rp * theta
psi_dot_des   = 0

# your tested mapping: omega_world = Ts(phi,theta,psi) @ euler_dot
Ts = Transformation.euler_to_spatial_rate_T(phi, theta, psi)
euler_dot_des = ca.vertcat(phi_dot_des, theta_dot_des, psi_dot_des)
omega_des_rp  = Ts @ euler_dot_des                        # 3x1, desired world angular velocity

# residual in angular velocity space (consistent with geo Jw)
e_omega = x_world_dot[3:6] - omega_des_rp                 # 3x1

# only penalize roll and pitch behavior, do not fight yaw
e_rp = e_omega[0:2]                                       # 2x1

# regularization (keeps solution small, avoids base thrashing)
e_reg = nu                                       # (6+n) x 1

# weighted least squares objective
w_rp  = ca.SX.sym('w_rp')                        # scalar
w_reg = ca.SX.sym('w_reg')                       # scalar
w_axis = ca.SX.sym('w_axis')  # scalar
cost = (
    ca.dot(e_p_task, e_p_task)
    + w_rp  * ca.dot(e_rp, e_rp)
    + w_reg * ca.dot(e_reg, e_reg)
    + w_axis * ca.dot(e_axis_task, e_axis_task)
)

# 1. Compute Hessian and Full Gradient
H, g_full = ca.hessian(cost, nu)

# 2. Extract the linear term of the gradient
# The full gradient is (H * nu + g_linear). 
# By substituting nu=0, we isolate g_linear.
g_linear = ca.substitute(g_full, nu, ca.DM.zeros(nu.shape))

# 3. Solve for the optimal nu
# Analytic solution for QP: H * nu + g_linear = 0  =>  nu = -inv(H) * g_linear
lam = 1e-8 + 1e-6 * w_reg
nu_star = -ca.solve(H + lam * ca.SX.eye(H.size1()), g_linear)

e_p_task_star = Jp @ nu_star - v_des
e_axis_task_star = Jw @ nu_star - w_des_axis

dt = ca.SX.sym('dt')

# linear part integrates directly
p_world_next = world_pose[0:3] + nu_star[0:3] * dt

# convert omega -> euler_dot for integration
psi = world_pose[5]
Ts = Transformation.euler_to_spatial_rate_T(world_pose[3], world_pose[4], psi)
euler_dot_star = ca.solve(Ts, nu_star[3:6])               # 3x1

rpy_next = world_pose[3:6] + euler_dot_star * dt

x_world_ref_next = ca.vertcat(p_world_next, rpy_next)
q_ref_next       = q + nu_star[6:6+n_joints] * dt

# 4. Create the function
# Now nu_star depends only on parameters (kp, p_des, etc.), not on the symbolic 'nu'
ik_local_func = ca.Function(
    'task_based_ik_local',
    [kp, p_des, q, w_rp, w_reg, k_rp, a_des, k_axis, w_axis, dt, world_pose, base_pose, tip_offset_pose],
    [x_world_ref_next, q_ref_next, e_p_task_star, e_axis_task_star]
)

q_new = q
world_pose_new = world_pose
for i in range(5000):
    x_world_next, q_next, e_p_task_star_new, e_axis_task_star_new = ik_local_func(kp, p_des, q_new, w_rp, w_reg, k_rp, a_des, k_axis, w_axis, dt, world_pose_new, base_pose, tip_offset_pose)
    q_new = q_next
    world_pose_new = x_world_next


ik_func = ca.Function('mapacc_task_based_ik', [q, world_pose, kp, p_des, w_rp, w_reg, k_rp, a_des, k_axis, w_axis, dt, base_pose, tip_offset_pose], [x_world_next, q_next, e_p_task_star_new])

In [4]:
ik_func.save('ik_whole_b.casadi')